In [ ]:
# pulls from the rotowire

In [1]:
# import the libraries
import requests
import pandas as pd
import json
import glob
from IPython.display import display
from datetime import datetime
import nfl_data_py as nfl

In [4]:
# this imports all modified columns correction and in order 

# define the template URL for API fetching
rotowire_template_url = "https://www.rotowire.com/betting/nfl/tables/nfl-games.php?week={week}"

# define the columns
columns_to_keep = ['gameID', 'year', 'week', 'gameDate', 'gameDateTime', 'homeAway', 'nickname', 'abbr',
                   'oppNickname', 'draftkings_moneyline', 'draftkings_spread', 'draftkings_ou', 'draftkings_score', 
                   'draftkings_oppScore', 'fanduel_moneyline', 'fanduel_spread', 'fanduel_ou', 'fanduel_score', 'fanduel_oppScore',
                   'betrivers_moneyline', 'betrivers_spread', 'betrivers_ou', 'betrivers_score', 'betrivers_oppScore',
                   'mgm_moneyline', 'mgm_spread', 'mgm_ou', 'mgm_score', 'mgm_oppScore'
                  ]

# fetch JSON data from API and filter columns
def import_rotowire_data(week):
    # Format the URL with the given week
    url = rotowire_template_url.format(week=week)
    
    # Fetch the JSON data from the API
    response = requests.get(url)
    
    if response.status_code == 200:
        # Convert the JSON data to a Python object
        json_data = response.json()
        
        if not json_data:
            return None
        
        # Convert the JSON data to a DataFrame
        df = pd.DataFrame(json_data)
        
        # Add the 'week' column manually
        df['week'] = week
        
        # Add the 'year' column derived from the 'gameDate' column
        df['gameDate'] = pd.to_datetime(df['gameDate'], errors='coerce')  # Convert to datetime
        df['year'] = df['gameDate'].dt.year
        
        # Drop all columns that are not in 'columns_to_keep'
        filtered_df = df[columns_to_keep]

        return filtered_df
    else:
        return None

# load missing weeks from local JSON files
def load_json_files():
    # List to hold data for missing weeks
    missing_weeks_data = []

    # Find all JSON files that follow the pattern for week files
    json_files = glob.glob("rotowire_odds_lines_week_*.json")
    
    for json_file in json_files:
        # Extract the week number from the file name
        week_number = int(json_file.split('_')[-1].replace('.json', ''))
        
        # Read the JSON file
        with open(json_file, 'r') as f:
            json_data = json.load(f)
        
        # Convert the JSON data to a DataFrame
        df = pd.DataFrame(json_data)
        
        # Add the week number and derive the year from the gameDate
        df['week'] = week_number
        df['gameDate'] = pd.to_datetime(df['gameDate'], errors='coerce')
        df['year'] = df['gameDate'].dt.year
        
        # Drop all columns that are not in 'columns_to_keep'
        filtered_df = df[columns_to_keep]
        missing_weeks_data.append(filtered_df)
    
    return missing_weeks_data

In [6]:
# fetch team totals and O/U data for multiple weeks (imports json files)
def teamTotals_overUnder():
    all_weeks_data = []

    # fetch data for each week (you can adjust the range as needed)
    for week in range(1, 18):  # adjust range based on the weeks you want to fetch
        df = import_rotowire_data(week)
        if df is not None:
            all_weeks_data.append(df)

    # load missing weeks from local JSON files and append them
    missing_weeks_data = load_json_files()
    all_weeks_data.extend(missing_weeks_data)

    # concatenate all non-empty DataFrames
    if all_weeks_data:
        full_season_data = pd.concat(all_weeks_data, ignore_index=True)

        # sort by 'year' (descending) and 'week' (ascending)
        full_season_data = full_season_data.sort_values(by=['year', 'week'], ascending=[False, True])

        # display the sorted DataFrame
        display(full_season_data)

        # save the sorted DataFrame to a CSV file
        csv_filename = "teamTotals_overUnder.csv"
        full_season_data.to_csv(csv_filename, index=False)
        print(f"Data successfully written to {csv_filename}")
    else:
        print("No data fetched for any week.")

# Call the function to execute the workflow
# teamTotals_overUnder()

In [9]:
# fetch team totals and O/U data for the current week only
def get_current_week():
    
    # Logic to compute the current NFL week
    current_date = datetime.now()
    
    # Define the start date of Week 1 in the NFL season (adjust this as needed)
    season_start_date = datetime(2024, 9, 4)
    
    # Calculate the week number based on the difference from the start date
    current_week = ((current_date - season_start_date).days // 7) + 1
    return current_week

def teamTotals_overUnder_current_week():
    current_week_data = []

    # Define the current week (you can adjust this to get the actual current week dynamically)
    current_week = get_current_week() 

    # Fetch data for the current week
    df = import_rotowire_data(current_week)
    if df is not None:
        current_week_data.append(df)

    # Concatenate all non-empty DataFrames
    if current_week_data:
        full_current_week_data = pd.concat(current_week_data, ignore_index=True)

        # Sort by 'year' (descending) and 'week' (ascending)
        full_current_week_data = full_current_week_data.sort_values(by=['year', 'week'], ascending=[False, True])

        # Display the sorted DataFrame
        display(full_current_week_data)

        # Save the sorted DataFrame to a CSV file
        csv_filename = "teamTotals_overUnder_current_week.csv"
        full_current_week_data.to_csv(csv_filename, index=False)
        print(f"Data successfully written to {csv_filename}")
    else:
        print("No data fetched for the current week.")

# Call the new function to fetch data for the current week
teamTotals_overUnder_current_week()

,gameID,year,week,gameDate,gameDateTime,homeAway,nickname,abbr,oppNickname,draftkings_moneyline,...,betrivers_moneyline,betrivers_spread,betrivers_ou,betrivers_score,betrivers_oppScore,mgm_moneyline,mgm_spread,mgm_ou,mgm_score,mgm_oppScore
0,2656699,2024,4,2024-09-29 13:00:00,Sep 29 1:00pm,away,Saints,NO,Falcons,124,...,125,3.0,41.0,19.0,22.0,130,3.0,41.0,19.0,22.0
1,2656699,2024,4,2024-09-29 13:00:00,Sep 29 1:00pm,home,Falcons,ATL,Saints,-148,...,-150,-3.0,41.0,22.0,19.0,-155,-3.0,41.0,22.0,19.0
2,2656706,2024,4,2024-09-29 13:00:00,Sep 29 1:00pm,away,Bengals,CIN,Panthers,-218,...,-210,-4.5,47.5,26.0,21.5,-225,-4.5,47.5,26.0,21.5
3,2656706,2024,4,2024-09-29 13:00:00,Sep 29 1:00pm,home,Panthers,CAR,Bengals,180,...,170,4.5,47.5,21.5,26.0,180,4.5,47.5,21.5,26.0
4,2656709,2024,4,2024-09-29 13:00:00,Sep 29 1:00pm,away,Rams,LAR,Bears,124,...,132,3.0,40.5,18.8,21.8,130,3.0,40.5,18.8,21.8
5,2656709,2024,4,2024-09-29 13:00:00,Sep 29 1:00pm,home,Bears,CHI,Rams,-148,...,-159,-3.0,40.5,21.8,18.8,-155,-3.0,40.5,21.8,18.8
6,2656713,2024,4,2024-09-29 13:00:00,Sep 29 1:00pm,away,Vikings,MIN,Packers,124,...,118,3.0,44.0,20.5,23.5,130,3.0,44.5,20.8,23.8
7,2656713,2024,4,2024-09-29 13:00:00,Sep 29 1:00pm,home,Packers,GB,Vikings,-148,...,-143,-3.0,44.0,23.5,20.5,-155,-3.0,44.5,23.8,20.8
8,2656718,2024,4,2024-09-29 13:00:00,Sep 29 1:00pm,away,Jaguars,JAX,Texans,195,...,215,6.0,44.5,19.3,25.3,220,6.0,44.5,19.3,25.3
9,2656718,2024,4,2024-09-29 13:00:00,Sep 29 1:00pm,home,Texans,HOU,Jaguars,-238,...,-275,-6.0,44.5,25.3,19.3,-275,-6.0,44.5,25.3,19.3


Data successfully written to teamTotals_overUnder_current_week.csv


In [12]:
# Function to pull QB data for the most recent previous week
def fetch_qb_data():
    # Define the columns you want to retrieve
    # qb_specific_columns = ['season', 'season_type', 'week', 'player_id', 
    #                        'player_name', 'position', 'recent_team']
    
    qb_specific_columns = ['week', 'player_id', 'player_name', 'position', 'recent_team']

    # Logic to determine the current week (replace with actual calculation as needed)
    season_start_date = datetime(2024, 9, 4)
    current_date = datetime.now()
    
    # Calculate the current week number based on the difference in days
    current_week = ((current_date - season_start_date).days // 7) + 1

    # Calculate the previous week
    previous_week = current_week - 1

    # Import data for the current season (2023) for the specified columns
    all_data = nfl.import_weekly_data(
        years=[2024],  # Current season, adjust if needed
        columns=qb_specific_columns
    )

    # Filter to show only QB positions
    # Filter to show only QB positions and only for the previous week
    qb_data = all_data[(all_data['position'] == 'QB') & (all_data['week'] == previous_week)]
    display(qb_data)

# Call the function to pull and display QB data week by week
fetch_qb_data()

Downcasting floats.


,week,player_id,player_name,position,recent_team
2,3,00-0023459,A.Rodgers,QB,NYJ
5,3,00-0026498,M.Stafford,QB,LA
7,3,00-0027973,A.Dalton,QB,CAR
11,3,00-0029604,K.Cousins,QB,ATL
31,3,00-0030565,G.Smith,QB,SEA
41,3,00-0031280,D.Carr,QB,NO
50,3,00-0031503,J.Winston,QB,CLE
68,3,00-0031800,T.Heinicke,QB,LAC
104,3,00-0033077,D.Prescott,QB,DAL
111,3,00-0033106,J.Goff,QB,DET


In [ ]:
## task ##
# build a new dataframe to insert a QB column into the output of the teamTotals_overUnder_current_week() function
# use the output of the fetch_qb_data() function to extract the QB and team
# you will need to match the QBs based on the 'recent_team' and 'abbr' columns. 